# ワクワクメールのデータ分析

In [1]:
import pandas as pd
import datetime
import seaborn as sns
import japanize_matplotlib

In [2]:
TOKYO_FILE_PATH = './../rawdata/wakuwaku_tokyo.csv'
KANAGAWA_FILE_PATH = './../rawdata/wakuwaku_kanagawa.csv'

tokyo_df = pd.read_csv(TOKYO_FILE_PATH)
kanagawa_df = pd.read_csv(KANAGAWA_FILE_PATH)

In [3]:
(len(tokyo_df), len(kanagawa_df))

(945, 467)

In [4]:
df_master = pd.merge(tokyo_df, kanagawa_df, how="outer")
df = df_master.drop(["image_url", "url"], axis=1)

In [5]:
print(len(df))
# df.head()

1412


## データ分析

### 地域別

In [6]:
output = pd.DataFrame(df.groupby("city").count()["id"])
output.index.name = "地域"
output = output.rename(columns = {"id": "投稿数"})
output.sort_values("投稿数", ascending=False).head(10)

,投稿数
地域,
新宿区,273
豊島区,189
横浜市西区,75
横浜市中区,42
台東区,42
足立区,35
渋谷区,35
川崎市川崎区,26
品川区,26


### 年代別

In [7]:
output = pd.DataFrame(df.groupby("age").count()["id"])
output.index.name = "年代"
output = output.rename(columns = {"id": "投稿数"})
output.sort_values("投稿数", ascending=False).head(20)

,投稿数
年代,
20代前半,521
20代半ば,225
20代後半,166
18-19歳,153
30代前半,135
30代半ば,66
30代後半,51
40代前半,33
40代半ば,27


### 時間帯別

In [8]:
def to_datetime(x):
    try:
        return datetime.datetime.strptime(x, '%Y/%m/%d %H:%M')                                        
    except Exception:
        now = datetime.datetime.now()
        post_at = datetime.datetime.strptime(x, '%m/%d %H:%M')
        if now.month == 1 and post_at.month == 12:
            post_at = post_at.replace(year=now.year - 1)
        else:
            post_at = post_at.replace(year=now.year)
        return post_at

output = df.assign(datetime=df.apply(lambda x: to_datetime(x.post_at), axis=1))
output = pd.DataFrame(output.groupby(output.datetime.map(lambda t: t.hour)).count()["id"])
output.index.name = "時間帯"
output = output.rename(columns = {"id": "投稿数"})
output.head(24)

,投稿数
時間帯,
0,36
1,34
2,24
3,21
4,13
5,19
6,10
7,20
8,33


### カテゴリ別

In [9]:
output = pd.DataFrame(df.groupby("genre").count()["id"])
output.index.name = "カテゴリ"
output = output.rename(columns = {"id": "投稿数"})
output.sort_values("投稿数", ascending=False).head(7)

,投稿数
カテゴリ,
すぐ会いたい,1198
今日じゃないけど...,40
大人の恋人候補,40
ミドルエイジ,39
アブノーマル,38
アダルト・H,37
既婚者,20


## 検索

### キーワード検索

In [10]:
#keyword = "会いたい"
#df[df['title'].str.contains(keyword)]

### 地域検索

In [11]:
area = "川崎市中原区"
df[df['city'].str.contains(area)]

,age,city,genre,id,name,post_at,prefecture,title
951,40代前半,川崎市中原区,既婚者,135271732,ﾁｬｲ,2019/11/18 0:14,神奈川県,三桁ポチャの私と不倫しませんか？
986,40代後半,川崎市中原区,アダルト・H,136649400,ななみ,1/1 15:26,神奈川県,サクッとエッチな事で会える車ある方待ってます
1051,40代後半,川崎市中原区,ミドルエイジ,136647507,ななみ,1/1 13:55,神奈川県,車ある人エッチな事でサクッと会えますよ
1054,40代半ば,川崎市中原区,ミドルエイジ,136633157,めぐみ,2019/12/31 23:51,神奈川県,見てくれた？約束しましょう
1061,40代半ば,川崎市中原区,ミドルエイジ,136607300,ちゃみ,2019/12/30 23:39,神奈川県,深夜25時くらいに
1064,40代半ば,川崎市中原区,ミドルエイジ,136574168,ちゃみ,2019/12/29 20:48,神奈川県,深夜25時くらいに
1067,40代半ば,川崎市中原区,ミドルエイジ,136541043,ちゃみ,2019/12/28 23:48,神奈川県,深夜25時くらいに
1075,40代半ば,川崎市中原区,すぐ会いたい,136649481,めぐみ,1/1 15:31,神奈川県,内容みてね。します？今日限定メール待ってます
1120,40代後半,川崎市中原区,すぐ会いたい,136629585,ななみ,2019/12/31 23:51,神奈川県,今から徒歩じゃない人、エッチな事で会えますよ！
1129,20代前半,川崎市中原区,すぐ会いたい,136633446,えな,2019/12/31 21:10,神奈川県,こんにちは～\r\n今から、武蔵小杉の近くに会いたい


## サクラ判定

In [12]:
eval_df = df_master

### 年齢によるフィルター

In [13]:
old_age_list = ["40代前半", "40代半ば", "40代後半", "50代前半", "50代半ば", "60代前半", "50代後半"]
eval_df = eval_df[eval_df.age.str[:3].astype(int) < 40]

### NGワードによるフィルター

In [14]:
ng_words = ["ニューハーフ", "下着", "生", "ftm", "MTF", "ビッチ", "お願い", 
            "イラマ", "オネエ", "痴漢", "男の娘", "車", "バイ",
            "おなべ", "ハメ撮り", "はめ撮り", "初体験", "処女", "童貞"]

def proc(x):
    for ng_word in ng_words:
        if ng_word in x:
            return True
    return False

mask = eval_df["title"].map(proc).map(lambda x: not x)
eval_df = eval_df[mask]

### 長すぎる名前のフィルター

In [15]:
def proc(x):
    if type(x) == str:
        return len(x) < 10
    else:
        return True

mask = eval_df["name"].map(proc)
eval_df = eval_df[mask]

### タイトルの出現頻度
タイトルの出現頻度による評価。頻度もさらに2回の頻度、3回以上の頻度で細分化できる。

In [16]:
duplicated_title_df = pd.DataFrame(eval_df.groupby("title").count())
output = duplicated_title_df.sort_values("id", ascending=False)

In [17]:
output = pd.DataFrame(output["id"])
output.index.name = "title"
output = output.rename(columns = {"id": "title_freq"})

In [18]:
eval_df = pd.merge(eval_df, output, on="title", how="left")

### プロフィールの出現頻度

In [19]:
# eval_df["name_age_city"] = eval_df["name"] + "_" + eval_df["age"] + "_" + eval_df["city"]
eval_df["name_age_city"] = eval_df["name"] + "_" + eval_df["age"]
output = pd.DataFrame(eval_df.groupby("name_age_city").count())

In [20]:
output = pd.DataFrame(output["id"])
output.index.name = "name_age_city"
output = output.rename(columns = {"id": "profile_freq"})

In [21]:
output[output.profile_freq == 1]

,profile_freq
name_age_city,
@*_ 20代前半,1
Ai_ 20代半ば,1
Alice_ 30代前半,1
ERIKA_ 20代半ば,1
Jun_ 20代前半,1
...,...
雨宮_ 20代半ば,1
麻友_ 30代前半,1
ｱﾐ_ 30代前半,1


In [22]:
eval_df = pd.merge(eval_df, output, on="name_age_city", how="left")

## 結果

In [23]:
result = eval_df
result = result.assign(datetime=result.apply(lambda x: to_datetime(x.post_at), axis=1))
result = result.sort_values("datetime")

one_day_ago = datetime.datetime.now() - datetime.timedelta(days=1)
result = result[result.datetime > one_day_ago]

result = result[(result.profile_freq == 1) & (result.title_freq == 1) ]

In [24]:
out = result.drop(["url", "image_url", "title_freq", "profile_freq", "name_age_city", "datetime"], axis=1)

In [25]:
len(out)

111

In [28]:
out_file_name = "wakuwaku_out.csv"
out.to_csv("./../data/" + out_file_name)